# Imports and function definitions

In [ ]:
# Used Python 3.7, beware

import pandas as pd
import numpy as np

data_case_storage="C://Users//Antoine//Documents//Ecole des Ponts//Cours 3A//Oliver Wymann//stored_data_case.h5"
parkmeter_coords_path = 'C://Users//Antoine//Documents//Ecole des Ponts//Cours 3A//Oliver Wymann//parkmeter_coords.h5'

In [ ]:
def list_HDF_file(file_name):
    with pd.HDFStore(file_name, complevel=9, complib='blosc') as store:
        result = store.keys()   
    return result

def read_HDF_file(file_name, table):
    with pd.HDFStore(file_name, complevel=9, complib='blosc') as store:
        return store[table]

# Data reading

In [ ]:
# List of the data series in this dataframe
print(list_HDF_file(data_case_storage))

In [ ]:
# Reading the file
df=read_HDF_file(data_case_storage, "/transaction_and_locations")

In [ ]:
print(df.keys())
print("Number of parkmeters : " + str(df["parkmeter_id"].drop_duplicates().count()))

# Data extraction and saving

In [ ]:
# Extracting the parkmeter coordinates
# The operation also retrieves the amount of money each parkmeter got
parkmeter_infos = df[['parkmeter_id', 'parkmeter_coordinates', 'amount']]
parkmeter_infos = parkmeter_infos.groupby(["parkmeter_id","parkmeter_coordinates"], as_index=False)["amount"].sum()
#print(parkmeter_infos)

In [ ]:
# Printing stuff
print(type(parkmeter_infos))
print("Shape of the dataframe : " + str(parkmeter_infos.shape) + "\n")

print("Ten first lines")
print(parkmeter_infos.head(10))
print("\nExample for one cell")
print(parkmeter_infos.iloc[0])

In [ ]:
# Storing the coordinates
parkmeter_infos.to_hdf(parkmeter_coords_path, key='parkmeter_coordinates')